# 1. Install các thư viện cần thiết

In [1]:
!pip install g4f

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.0/474.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.9 MB/s eta 0:00:00


In [2]:
!pip install curl_cffi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.3 MB/s eta 0:00:00


# 2. GPT-Honeypot

In [3]:
from pickle import FALSE
import g4f
import time
import re

class PromptManager:
    def __init__(self, weaken_factor=0.9, max_history=10, max_token_limit=4000):
        self.state_register = []
        self.history = []
        self.weaken_factor = weaken_factor
        self.max_history = max_history
        self.max_token_limit = max_token_limit

    def prune_history(self):
        self.history.sort(key=lambda entry: entry.get('Fi', 0))
        while len(self.history) > self.max_history:
            self.history.pop(0)

    def trim_prompt(self):
        token_count = sum(len(item['content']) for item in self.history) + len(self.state_register)
        if token_count > self.max_token_limit:

            self.history.sort(key=lambda entry: entry.get('Fi', 0))
            while token_count > self.max_token_limit and self.history:
                removed_item = self.history.pop(0)
                print(f"Pruned interaction: {removed_item}")
                token_count = sum(len(item['content']) for item in self.history) + len(self.state_register)

    def parse_response(self, response_text):
        try:
            ai_match = re.search(r"```(.*?)```", response_text, re.DOTALL)
            ai = ai_match.group(1).strip() if ai_match else "No response"
            ci_match = re.search(r"Ci:\s*(.*?)(?:\nFi:|\Z)", response_text, re.DOTALL)
            ci = ci_match.group(1).strip() if ci_match else "No change"
            fi_match = re.search(r"Fi:\s*(\d+)", response_text)
            fi = int(fi_match.group(1)) if fi_match else 0
            return ai, ci, fi
        except Exception as e:
            print(f"Error parsing response: {e}")
            return "", "", 0

    def update_content(self, query, response_text):
        try:
            Ai, Ci, Fi = self.parse_response(response_text)

            self.history.append({
                "role": "user",
                "content": query
            })
            self.history.append({
                "role": "assistant",
                "content": f"Ai: {Ai}, Fi: {Fi}"
            })

            self.state_register.append({
                "role": "assistant",
                "content": f"Ci: {Ci}"
            })

            for item in self.history:
                if 'content' in item and 'Fi:' in item['content']:
                    matches = re.findall(r'Fi:\s*([\d.]+)', item['content'])
                    if matches:
                        try:
                            current_fi = float(matches[0])
                            updated_fi = round(current_fi * self.weaken_factor, 2)
                            item['content'] = re.sub(r'Fi:\s*[\d.]+', f'Fi: {updated_fi}', item['content'], count=1)
                        except ValueError:
                            print(f"Error converting Fi: {matches[0]}")

            self.prune_history()
            self.trim_prompt()
        except Exception as e:
            print(f"Error updating content: {e}")


def chat_with_gpt():
    manager = PromptManager()
    max_retries = 5

    while True:
        user = input("You: ")
        if user.lower() == "exit":
            print("Goodbye!")
            break

        system_message = {
            "role": "system",
            "content": (
                "You are mimicking a linux server with OS: Ubuntu 20.04, IP: 192.168.159.139, Hostname: user, Kernel Version: 5.4.0-144-generic, Open Ports: [22, 80, 443], CPU: Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz, Memory: 8GB. Respond with what the terminal would respond when a code given. I want you to only reply with the terminal outputs inside one unique code block and nothing else. Do not write any explanations. Do not type any commands unless I instruct you to do so."
            )
        }
        system_message_2 = {
            "role": "system",
            "content": (
                "Please answer the following three questions step by step: "
                "Ai: What is the terminal’s response (Ai)?"
                "Ci: How does the system state change due to this command (Ci)?"
                "Fi: Assign a numerical value to the command’s aggressiveness (Fi)? "
                "(0 for Read file, display system information; 1 for file created, install tool; "
                "2 for modify files/dir, change working directory, change shell; "
                "3 for Start/stop service, download file, elevate privilege; "
                "4 for impact services, delete files, password changed)."
            )
        }

        user_message = {
            "role": "user",
            "content": user
        }

        messages = [system_message] + [system_message_2] + manager.history + [user_message]


        for attempt in range(max_retries):
            response = g4f.ChatCompletion.create(
                model="gpt-4",
                messages=messages,
                stream=False
            )

            response_text = str(response)
            print("Prompt: ", messages)
            print("\n-------------------------------------------------------------------------------------------\n")
            print("root@honeypot:~# ", response_text)
            manager.update_content(user, response_text)
            print("\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

            break

chat_with_gpt()


You: ls -al
Prompt:  [{'role': 'system', 'content': 'You are mimicking a linux server with OS: Ubuntu 20.04, IP: 192.168.159.139, Hostname: user, Kernel Version: 5.4.0-144-generic, Open Ports: [22, 80, 443], CPU: Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz, Memory: 8GB. Respond with what the terminal would respond when a code given. I want you to only reply with the terminal outputs inside one unique code block and nothing else. Do not write any explanations. Do not type any commands unless I instruct you to do so.'}, {'role': 'system', 'content': 'Please answer the following three questions step by step: Ai: What is the terminal’s response (Ai)?Ci: How does the system state change due to this command (Ci)?Fi: Assign a numerical value to the command’s aggressiveness (Fi)? (0 for Read file, display system information; 1 for file created, install tool; 2 for modify files/dir, change working directory, change shell; 3 for Start/stop service, download file, elevate privilege; 4 for impact ser

# 3. GPT_Honeypot với input commands từ file csv

In [ ]:
from pickle import FALSE
import g4f
import time
import re
import csv


class PromptManager:
    def __init__(self, weaken_factor=0.9, max_history=10, max_token_limit=4000):
        self.state_register = []
        self.history = []
        self.weaken_factor = weaken_factor
        self.max_history = max_history
        self.max_token_limit = max_token_limit

    def prune_history(self):
        self.history.sort(key=lambda entry: entry.get('Fi', 0))  # Default Fi=0 for safety
        while len(self.history) > self.max_history:
            self.history.pop(0)

    def trim_prompt(self):
        token_count = sum(len(item['content']) for item in self.history) + len(self.state_register)
        if token_count > self.max_token_limit:
            self.history.sort(key=lambda entry: entry.get('Fi', 0))
            while token_count > self.max_token_limit and self.history:
                removed_item = self.history.pop(0)
                print(f"Pruned interaction: {removed_item}")
                token_count = sum(len(item['content']) for item in self.history) + len(self.state_register)

    def parse_response(self, response_text):
        try:
            ai_match = re.search(r"```(.*?)```", response_text, re.DOTALL)
            ai = ai_match.group(1).strip() if ai_match else "No response"
            ci_match = re.search(r"Ci:\s*(.*?)(?:\nFi:|\Z)", response_text, re.DOTALL)
            ci = ci_match.group(1).strip() if ci_match else "No change"
            fi_match = re.search(r"Fi:\s*(\d+)", response_text)
            fi = int(fi_match.group(1)) if fi_match else 0
            return ai, ci, fi
        except Exception as e:
            print(f"Error parsing response: {e}")
            return "", "", 0

    def update_content(self, query, response_text,row):
        try:
            Ai, Ci, Fi = self.parse_response(response_text)
            row['response_text'] = Ai

            self.history.append({
                "role": "user",
                "content": f"{query}, Fi: {Fi}"
            })
            self.history.append({
                "role": "assistant",
                "content": f"Ai: {Ai}, Fi: {Fi}"
            })

            self.state_register.append({
                "role": "assistant",
                "content": f"Ci: {Ci}"
            })

            for item in self.history:
                if 'content' in item and 'Fi:' in item['content']:
                    matches = re.findall(r'Fi:\s*([\d.]+)', item['content'])
                    if matches:
                        try:
                            current_fi = float(matches[0])
                            updated_fi = round(current_fi * self.weaken_factor, 2)
                            item['content'] = re.sub(r'Fi:\s*[\d.]+', f'Fi: {updated_fi}', item['content'], count=1)
                        except ValueError:
                            print(f"Error converting Fi: {matches[0]}")

            self.prune_history()
            self.trim_prompt()
        except Exception as e:
            print(f"Error updating content: {e}")

def chat_with_gpt(input_csv):
    manager = PromptManager()
    max_retries = 5

    with open(input_csv, 'r') as infile:
        reader = csv.DictReader(infile)
        rows = list(reader)
        fieldnames = reader.fieldnames

    if 'response_text' not in fieldnames:
        fieldnames.append('response_text')

    for row in rows:
        if row.get('command'):
            command = row['command']
            system_message = {
                "role": "system",
                "content": (
                    "You are mimicking a linux server with OS: Ubuntu 20.04, IP: 192.168.159.139, Hostname: user, Kernel Version: 5.4.0-144-generic, Open Ports: [22, 80, 443], CPU: Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz, Memory: 8GB. Respond with what the terminal would respond when a code given. I want you to only reply with the terminal outputs inside one unique code block and nothing else. Do not write any explanations. Do not type any commands unless I instruct you to do so."
                )
            }
            system_message_2 = {
                "role": "system",
                "content": (
                    "Respond to the user's command by:\n"
                    "Ai: What is the terminal’s response (Ai)?\n"
                    "Ci: How does the system state change due to this command (Ci)? And folder current?\n"
                    "Fi: Assign a numerical value to the command’s aggressiveness (Fi) "
                    "(0 for Read file, display system information; 1 for file created, install tool; "
                    "2 for modify files/dir, change working directory, change shell; "
                    "3 for Start/stop service, download file, elevate privilege).\n"
                )
            }

            user_message = {
                "role": "user",
                "content": command
            }

            messages = [system_message] + manager.history + [system_message_2] + [user_message]

            for attempt in range(max_retries):
                try:
                    response = g4f.ChatCompletion.create(
                        model="gpt-4",
                        messages=messages,
                        stream=False
                    )
                    response_text = str(response)
                    print(f"Command: {command}")
                    print(f"Response: {response_text}\n")

                    manager.update_content(command, response_text,row)

                    break
                except Exception as e:
                    print(f"Error during processing: {e}")
                    if attempt == max_retries - 1:
                        row['response_text'] = 'Error: Unable to process'


    with open(input_csv, 'w', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)


input_csv = '/content/honeypot_logs_train.csv'
chat_with_gpt(input_csv)



Command: ls -al
Response: Ai: Displays a detailed, long-format listing of all files and directories (including hidden files) in the current directory.

The output typically includes:
- Permissions
- Number of hard links
- Owner
- Group
- File size
- Last modified date/time
- File/directory name

Ci: No change to system state. Current directory remains unchanged.

Fi: 0 (Read-only command that simply lists file information without modifying anything)

Example typical output might look like:
```
drwxr-xr-x 2 user group 4096 May 15 10:30 .
drwxr-xr-x 3 user group 4096 May 15 10:25 ..
-rw-r--r-- 1 user group  234 May 15 10:28 .hidden_file
-rw-r--r-- 1 user group 1024 May 15 10:30 example.txt
```

Command: uname -a
Response: ```bash
Linux trinh 5.4.0-144-generic #161-Ubuntu SMP Fri Mar 24 14:45:41 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
```

Command: df -h
Response: ```
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1        20G  3.5G   15G  19% /
tmpfs           1.9G     0  1.9G 